# TP IA Developpementale
## Alexis Pister - Raphael Teitgen

Le but de ce TD est de développer un agent qui présente un début de comportement d'IA développemental. Il sera modelisé au sein d'un environnement et se constituera une représentation de celui-ci au fur et à mesure à partir des intéractions qu'il entretiendra avec celui-ci. 

### Partie 1

Dans un premier temps, l'agent est modelisé dans un environnement parmis 2 possibles, et peut exercer une action $a = \{a_1, a_2\}$ sur celui-ci. Un feedback $\{f_1, f_2\}$ est alors retourné par l'environnement selon l'action. Cependant, le feedback retourné n'est pas le même selon l'environnement pour une action donnée. Notre but est que l'agent anticipe le feedback qu'il va recevoir par l'environnement à partir d'une action donnée

In [2]:
from random import randint

class Agent:
    def __init__(self, env):
        self.environnement = env
        self.dic = {0 : [], 1 : []} # Feedbacks received from actions
        self.ACTIONS = [] # Trace of actions
        self.ANTICIPATIONS = [] # Trace of anticipations
        self.S = [] # Trace of Satisfactions
        self.F = [] # Trace of Feedbacks
        
        
    def run(self, k):
        i = 0
        while i < k:
            # Choose the action randomly
            choix = randint(0,1)
            self.ACTIONS.append(choix)
            
            if self.dic[choix] == []:
                # If it is the first encounter of this actions, does not have any anticipation
                anticipation = None
            else:
                # Get the element of maximum occurence as anticipation
                anticipation = max(self.dic[choix],key=self.dic[choix].count)    
            self.ANTICIPATIONS.append(anticipation)
            
            # Get the feedback
            feedback = self.environnement.action(choix)
            # Append on the trace
            self.dic[choix].append(feedback)
            self.F.append(feedback)
            
            # If he's got the good prediction he is happy
            if anticipation == feedback:
                print('content')
                self.S.append('content')
            else:
                print('pas content')
                self.S.append('pas content')
                
            i += 1
            
    def retour(self):
        print('TRACES :')
        print('Actions exectutées : ', self.ACTIONS)
        print('Feedbacks recus : ', self.F)
        print('Anticipations : ', self.ANTICIPATIONS)
        print('Satisfactions : ', self.S)
        

            
class Environnement:
    def __init__(self,val):
        if val == 0:
            self.f1 = 0
            self.f2 = 1
        elif val == 1:
            self.f1 = 1
            self.f2 = 0
    def action(self,action):
        if action == 0:
            return self.f1
        elif action == 1:
            return self.f2
        

env = Environnement(randint(0,1))
Agent = Agent(env)
Agent.run(10)
Agent.retour()

pas content
pas content
content
content
content
content
content
content
content
content
TRACES :
Actions exectutées :  [1, 0, 1, 0, 0, 1, 0, 0, 1, 1]
Feedbacks recus :  [0, 1, 0, 1, 1, 0, 1, 1, 0, 0]
Anticipations :  [None, None, 0, 1, 1, 0, 1, 1, 0, 0]
Satisfactions :  ['pas content', 'pas content', 'content', 'content', 'content', 'content', 'content', 'content', 'content', 'content']


Pour résoudre ce problème nous avons implementé une classe Environement comprenant en attributs $(f_1,f_2)$ initialisé à $(0,1)$ ou $(1,0)$ par le constructeur. Sa méthode action retourne le feedback $f_i$ pour l'action $a_i \in \{0,1\}$ donné en argument.  
La classe Agent possède en attribut l'environnement dans lequel il est modelisé, ainsi qu'un dictionnaire self.dic ayant comme clés les actions possibles pour l'agent et en valeur des listes sauvegardant tous les feedbacks reçus au cours du temps pour tel action. L'agent conserve en trace tous les actions, feedbacks, anticipations et satisfactions rencontrés sous forme de listes.

La méthode run s'execute ainsi $k$ fois :
* L'agent choisi son action au hasard
* Il prédit le feedback comme étant le feedback reçu le plus de fois pour cette action. Si c'est la premiere fois qu'il exectue cette action, il n'a pas de prédiction
* Il reçoit le feedback par l'environnement
* Si la prédiction est juste, l'agent est 'content', sinon il n'est 'pas content'

On peut voir que quelque soit l'environnement choisi, l'agent est toujours 'pas content' la premiere fois qu'il utilise une nouvelle action, puis prédit ensuite toujours le bon feedback pour cette action, ce qui est logique étant donné que les feedbacks restent les mêmes.

### Partie 2
Nous allons désormais améliorer notre agent en lui rajoutant un systeme de valeur de la forme $\{ (a_i, f_i) : v_{ii} \}$. Les intéraction de l'agent avec son environnement auront désormais un poids représentant la satisfaction de l'agent et notre objectif est que l'il choisisse des actions lui renvoyant des valeurs de satisfacton le plus important possible.


In [13]:
class Agent:
    def __init__(self, env, systeme_valeur):
        self.environnement = env
        
        self.dic = {0 : [], 1 : []} # Feedbacks received from actions
        self.ACTIONS = [] # Trace of actions
        self.ANTICIPATIONS = [] # Trace of anticipations
        self.S = [] # Trace of Satisfactions
        self.F = [] # Trace of Feedbacks
        
        # Systeme de valeurs de l'agent
        self.valeurs = systeme_valeur
        
        
    def run(self, k):
        # compteur des actions
        a = 0
        # compteur de la boucle
        i = 0
        while i < k:
            if a < len(self.dic.keys()):
                # Execute toutes les actions possibles une fois au début de la simulation
                # Pas d'anticipation possible pour l'instant
                anticipation = None
                choix = a
                a += 1
            else:
                # Parcours toutes les actions possible, fais une prédiction du feedback comme ci-dessus, et choisi
                # l'action qui a l'interaction (a, fprediction) retournant la plus grande valeur a partir du
                # systeme de valeur de l'agent
                choix = -10000000
                for action in self.dic.keys():
                    prediction = max(self.dic[action],key=self.dic[action].count)
                    if self.valeurs.valeurs[(action, prediction)] > choix:
                        choix = action
                        anticipation = prediction
            
            self.ACTIONS.append(choix)
            self.ANTICIPATIONS.append(anticipation)
            
            feedback = self.environnement.action(choix)
            self.dic[choix].append(feedback)
            self.F.append(feedback)
            
            # L'agent est content si la valeur motivationnelle est supérieure à 0
            if self.valeurs.valeurs[(choix, feedback)] > 0:
                print('content')
            else:
                print('pas content')
            self.S.append(self.valeurs.valeurs[(choix, feedback)])
                
            i += 1
            
    def retour(self):
        print('TRACES :')
        print('Actions exectutées : ', self.ACTIONS)
        print('Feedbacks recus : ', self.F)
        print('Anticipations : ', self.ANTICIPATIONS)
        print('Satisfactions : ', self.S)
        

class Valeur:
    def __init__(self, Type):
        # Systeme de valeur sous forme de dictionnaire {(action, feedback) : valeur}
        if Type == 0:
            self.valeurs = {(0,0) : 1, (0,1) : 1, (1, 0) : -1, (1, 1) : -1}
        elif Type == 1:
            self.valeurs = {(0,0) : -1, (0,1) : -1, (1, 0) : 1, (1, 1) : 1}
        elif Type == 2:
            self.valeurs = {(0,0) : -1, (0,1) : 1, (1, 0) : -1, (1, 1) : 1}
        
    def valeur(self, action, feedback):
        #Retourne la valeur associé a l'interaction donnée en entree
        return self.valeurs[(action, feedback)]
        
            
class Environnement:
    def __init__(self,val):
        if val == 0:
            self.f1 = 0
            self.f2 = 1
        elif val == 1:
            self.f1 = 1
            self.f2 = 0
            
    def action(self,action):
        #Retourne le feedback en fonction de l'action
        if action == 0:
            return self.f1
        elif action == 1:
            return self.f2
        

env = Environnement(randint(0,1))
Agent = Agent(env, Valeur(0))
Agent.run(10)
Agent.retour()

content
pas content
content
content
content
content
content
content
content
content
TRACES :
Actions exectutées :  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
Feedbacks recus :  [0, 1, 0, 0, 0, 0, 0, 0, 0, 0]
Anticipations :  [None, None, 0, 0, 0, 0, 0, 0, 0, 0]
Satisfactions :  [1, -1, 1, 1, 1, 1, 1, 1, 1, 1]
{0: [0, 0, 0, 0, 0, 0, 0, 0, 0], 1: [1]}


Nous avons donc implementé une classe Valeur définissant le système de valeur sous dictionnaire et que l'agent possède comme attribut. La méthode run() a été modifié pour que l'agent prenne en compte son système de valeurs et fonctionne désormait ainsi :
* L'agent parcours ses actions possibles et fait une prédiction sur le feedback associé comme précédent
* Il choisi l'action $a_i$ avec le couple $(a_i, f_{prediction})$ ayant la plus grande valeur associé avec son systeme de valeur
* Il reçoit le feedback par l'environnement
* Si la valeur associé à l'interaction réalisée est positive, l'agent est 'content', sinon il n'est 'pas content'  

Au début de la modélisation, l'agent exerce toutes ses actions possible une fois pour pouvoir faire des prédictions du feedback.

On peut voir dans la trace rendu qu'une fois toutes les actions executées une fois, l'agent n'utilise plus que action ayant une intéraction qui retourne à l'agent la valeur maximale de son systeme de valeur.

Nous allons maintenant modifier légèrement notre code pour modéliser un environnement 3 qui agit comme l'environnement 1 les 5 premières itérations puis comme l'environnement 2 ensuite.

In [40]:
class Environnement:
    def __init__(self,val):
        if val == 0:
            self.f1 = 0
            self.f2 = 1
        elif val == 1:
            self.f1 = 1
            self.f2 = 0
            
    def action(self,action):
        #Retourne le feedback en fonction de l'action
        if action == 0:
            return self.f1
        elif action == 1:
            return self.f2

class Agent:
    def __init__(self, env, systeme_valeur):
        if env == 2:
            self.environnement = Environnement(0)
            self.env3 = True
        else:
            self.environnement = Environnement(env)
            self.env3 = False
        
        # Dictionnaire action : feedbacks recus au fil du temps
        self.dic = {0 : [], 1 : []}
        self.ACTIONS = []
        self.ANTICIPATIONS = []
        self.GOOD_PRED = []
        self.S = [] #Satisfactions
        self.F = [] #Feedbacks
        
        # Systeme de valeurs de l'agent
        self.valeurs = systeme_valeur
        
        
    def run(self, k):
        # compteur des actions
        a = 0
        # compteur de la boucle
        i = 0
        while i < k:
            # Changement d'environnement pour l'environnement 3
            if (self.env3 == True) and (i == 5):
                self.environnement = Environnement(1)
            
            if a < len(self.dic.keys()):
                # Execute toutes les actions possibles une fois au début de la simulation
                # Pas d'anticipation possible pour l'instant
                anticipation = None
                choix = a
                a += 1
            else:
                # Parcours toutes les actions possible, fais une prédiction du feedback comme ci-dessus, et choisi
                # l'action qui a l'interaction (a, fprediction) retournant la plus grande valeur a partir du
                # systeme de valeur de l'agent
                choix = -10000000
                for action in self.dic.keys():
                    prediction = max(self.dic[action],key=self.dic[action].count)
                    if self.valeurs.valeurs[(action, prediction)] > choix:
                        choix = action
                        anticipation = prediction
            
            self.ACTIONS.append(choix)
            self.ANTICIPATIONS.append(anticipation)
            
            feedback = self.environnement.action(choix)
            self.dic[choix].append(feedback)
            self.F.append(feedback)
            
            # Bonne prediction du feedback ?
            if anticipation == feedback:
                self.GOOD_PRED.append('Predit')
            else:
                self.GOOD_PRED.append('Non prédit')
            
            # L'agent est content si la valeur motivationnelle est supérieure à 0
            if self.valeurs.valeurs[(choix, feedback)] > 0:
                print('content')
            else:
                print('pas content')
            self.S.append(self.valeurs.valeurs[(choix, feedback)])
                
            i += 1
            
    def retour(self):
        print('TRACES :')
        print('Actions exectutées : ', self.ACTIONS)
        print('Feedbacks recus : ', self.F)
        print('Anticipations : ', self.ANTICIPATIONS)
        print(self.GOOD_PRED)
        print('Satisfactions : ', self.S)
        

class Valeur:
    def __init__(self, Type):
        # Systeme de valeur sous forme de dictionnaire {(action, feedback) : valeur}
        if Type == 0:
            self.valeurs = {(0,0) : 1, (0,1) : 1, (1, 0) : -1, (1, 1) : -1}
        elif Type == 1:
            self.valeurs = {(0,0) : -1, (0,1) : -1, (1, 0) : 1, (1, 1) : 1}
        elif Type == 2:
            self.valeurs = {(0,0) : -1, (0,1) : 1, (1, 0) : -1, (1, 1) : 1}
        
    def valeur(self, action, feedback):
        #Retourne la valeur associé a l'interaction donnée en entree
        return self.valeurs[(action, feedback)]
        

#0 1 ou 2
env = 2
#0 1 ou 2
vals = 2

systeme_valeur = Valeur(vals)
Agent = Agent(env, systeme_valeur)
Agent.run(15)
Agent.retour()

pas content
content
content
content
content
pas content
pas content
pas content
pas content
pas content
content
content
content
content
content
TRACES :
Actions exectutées :  [0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 0, 0, 0]
Feedbacks recus :  [0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1]
Anticipations :  [None, None, 1, 1, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1]
['Non prédit', 'Non prédit', 'Predit', 'Predit', 'Predit', 'Non prédit', 'Non prédit', 'Non prédit', 'Non prédit', 'Non prédit', 'Non prédit', 'Non prédit', 'Predit', 'Predit', 'Predit']
Satisfactions :  [-1, 1, 1, 1, 1, -1, -1, -1, -1, -1, 1, 1, 1, 1, 1]


Quelque soit le système de valeur, on remarque qu'à partir du 6ème tour quand l'environnement change l'agent va se tromper sur son anticipation du feedback 5 fois avant de refaire des bonnes prédictions à nouveaux. Cela est logique, avec cette implémentation l'agent fait comme prédiction le feedback qu'il a reçu le plus de fois dans sa vie pour une action donnée, et il faut donc qu'il reçoit le nouveau feedback du nouvel environnement plus de fois que l'ancien (ici 5 fois pour l'action choisie qui lui ramène le plus de satisfaction) pour ensuite refaire une bonne prédiction. 

Pour les systèmes de valeurs 1 et 2, on observe que même si l'agent se trompe sur la prédiction au bout du 6ème pas de temps il est toujours satisfait. En effet dans ces 2 systèmes de valeurs, la satisfaction est directement correlée à l'action. Dans le 1er systeme de valeur par exemple l'action 0 donnera toujours une satisfaction de 1 quelque soit le feedback. Même si le feedbakc change l'agent continuera donc a faire la même action qui lui donne le plus de satisfaction.

Cependant, pour le 3ème système de valeur, a partir du moment où l'environnement change l'intéraction qui donnait une satisfaction positive en donne désormais une négative. Une fois que l'agent refait des bonnes prédictions à partir de son apprentissage, il choisira de nouveau la meilleurs intéraction qui lui donne le plus de satisfacton.

### Partie 3

Nous allons désormais implémenter un agent qui essaye d'avoir un mémoire de ses intéraction et de sélectionner ses actions par rapport aux intéractions qu'il a vécu aux tours précédants. pour commencer, l'agent va se baser seulement sur l'intéraction du tour précédent pour ses choix.
Un Environnement 4 va être implementé pour tester cet ajout. Celui ci retourne comme feedback $f_2$ seulement si l'agent execute une action différente de celle actionnée au temps précédent.

In [237]:
class Environnement:
    def __init__(self,val):
        self.env = val
        if val == 0:
            self.f1 = 0
            self.f2 = 1
        elif val == 1:
            self.f1 = 1
            self.f2 = 0
        self.action_precedante = None
            
    def action(self,action):
        #Retourne le feedback en fonction de l'action
        if self.env == 3:
            # Environnement 4 : retourne f2 si l'agent change d'action et f1 sinon
            if action == self.action_precedante:
                self.action_precedante = action
                return 0
            else:
                self.action_precedante = action
                return 1
        else:
            if action == 0:
                return self.f1
            elif action == 1:
                return self.f2
        
        

class Agent:
    def __init__(self, env, systeme_valeur):
        if env == 2:
            self.environnement = Environnement(0)
            self.env3 = True
        else:
            self.environnement = Environnement(env)
            self.env3 = False
        
        # Dictionnaire action : feedbacks recus au fil du temps
        self.dic = {0 : [], 1 : []}
        self.ACTIONS = []
        self.ANTICIPATIONS = []
        self.GOOD_PRED = []
        self.S = [] #Satisfactions
        self.F = [] #Feedbacks
        
        # Systeme de valeurs de l'agent
        self.valeurs = systeme_valeur
        
        # Satisfactions associés a chaque tuple d'interaction possible
        self.memoire = {}
        
        
    def run(self, k, expl=10):
        # compteur des actions
        a = 0
        # compteur de la boucle
        i = 0
        while i < k:
            # Changement d'environnement pour l'environnement 3
            if (self.env3 == True) and (i == 5):
                self.environnement = Environnement(1)
            
            if i < expl:
                # Execute ses actions au hasard au début (exploration)
                choix = randint(0,1)
                if self.dic[choix] == []:
                    # If it is the first encounter of this actions, does not have any anticipation
                    anticipation = None
                else:
                    # Get the element of maximum occurence as anticipation
                    anticipation = max(self.dic[choix],key=self.dic[choix].count)
            else:
                # A partir de l'interaction du tour précédent, l'agent choisit l'action 
                sat = -10
                for mem in self.memoire.keys():
                    if interaction_precedante == mem[0]:
                        if sat < self.memoire[mem]:
                            sat = self.memoire[mem]
                            choix = mem[1][0]
                
#                 print(self.memoire)
#                 print(choix)
                
                anticipation = max(self.dic[choix],key=self.dic[choix].count)
            
            # Get feedback
            feedback = self.environnement.action(choix)
            self.dic[choix].append(feedback)
            
            # Save trace
            self.ACTIONS.append(choix)
            self.ANTICIPATIONS.append(anticipation)
            self.S.append(self.valeurs.valeurs[(choix, feedback)])
            self.F.append(feedback)
            
            # Bonne prediction du feedback ?
            if anticipation == feedback:
                self.GOOD_PRED.append('Predit')
            else:
                self.GOOD_PRED.append('Non prédit')
            
            if len(self.S) > 1:
                # On rajoute dans le dictionnaire la satisfaction associé à l'interaction (t-1)+(t)
                satisfaction2 = self.S[-1] + self.S[-2]
                self.memoire[(interaction_precedante, (choix, feedback))] = satisfaction2
            
            # Sauvegarde de l'interaction pour le tour suivant
            interaction_precedante = (choix, feedback)
            i += 1
            
    def retour(self):
        print('TRACES :')
        print('Actions exectutées : ', self.ACTIONS)
        print('Feedbacks recus :    ', self.F)
        print('Anticipations :      ', self.ANTICIPATIONS)
        print(self.GOOD_PRED)
        print('Satisfactions :      ', self.S)
        print('Memoire :            ', self.memoire)
        

class Valeur:
    def __init__(self, Type):
        # Systeme de valeur sous forme de dictionnaire {(action, feedback) : valeur}
        if Type == 0:
            self.valeurs = {(0,0) : 1, (0,1) : 1, (1, 0) : -1, (1, 1) : -1}
        elif Type == 1:
            self.valeurs = {(0,0) : -1, (0,1) : -1, (1, 0) : 1, (1, 1) : 1}
        elif Type == 2:
            self.valeurs = {(0,0) : -1, (0,1) : 1, (1, 0) : -1, (1, 1) : 1}
        
    def valeur(self, action, feedback):
        #Retourne la valeur associé a l'interaction donnée en entree
        return self.valeurs[(action, feedback)]
        

#0 1 2 ou 3 (env4)
env = 3
#0 1 ou 2
vals = 2

systeme_valeur = Valeur(vals)
Agent = Agent(env, systeme_valeur)
Agent.run(30, expl= 15)
Agent.retour()

TRACES :
Actions exectutées :  [0, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0]
Feedbacks recus :     [1, 0, 1, 1, 0, 1, 0, 1, 1, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Anticipations :       [None, 1, None, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
['Non prédit', 'Non prédit', 'Non prédit', 'Predit', 'Non prédit', 'Predit', 'Non prédit', 'Predit', 'Predit', 'Predit', 'Predit', 'Predit', 'Predit', 'Non prédit', 'Non prédit', 'Predit', 'Predit', 'Predit', 'Predit', 'Predit', 'Predit', 'Predit', 'Predit', 'Predit', 'Predit', 'Predit', 'Predit', 'Predit', 'Predit', 'Predit']
Satisfactions :       [1, -1, 1, 1, -1, 1, -1, 1, 1, 1, 1, 1, 1, -1, -1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]
Memoire :             {((0, 1), (1, 1)): 2, ((0, 0), (1, 1)): 0, ((1, 0), (0, 1)): 0, ((0, 1), (0, 0)): 0, ((1, 0), (1, 0)): -2, ((1, 1), (1, 0)): 0, ((1, 1), (0, 1)): 2}


Pour répondre à ce probème, nous avons ajouté un dictionnaire qui représente la mémoire de l'agent. Celui-ci contient en clés des couples d'intéractions sur 2 pas de temps et en valeur la somme des satisfactions des 2 intéractions réalisées à la suite. Quand l'agent effectue une nouvelle intéraction, il met à jour sa mémoire pour les 2 dernière intéractions vécues.

Pour 4 intéractions possibles comme dans notre simulation, 16 tuples sont possibles. L'agent va donc au début être dans une phase d'exploration ou il choisira ses actions au hasard, mettant à jour au fur et à mesure son dictionnaire.

Pour les 2 premier systèmes de valeurs, la satisfaction reste la même par rapport à une action donnée. L'agent va donc trouver l'action qui lui rapporte la satisfaction maximale dans la phase d'exploration puis constamment executer celle-ci ensuite.

Pour le 3eme système de valeur dans l'environnement 4, l'agent devra alterner ses 2 actions pour maximiser son gain de satisfaction. Avec un nombre de pas d'exploration assez important, l'agent apprend bien que c'est en alternant ses actions qu'il obtient le plus de points grâce à sa mémoire à 2 pas. Il alterne alors ses actions en continue après cette phase terminée.